In [1]:
from scipy import spatial

def cosine(x, y):
    dataSetI = x
    dataSetII = y
    return 1 - spatial.distance.cosine(dataSetI, dataSetII)

In [2]:
import pickle

def load_category(category):
    pickle_in = open("elmo_category/"+ category +".pickle", "rb")
    return pickle.load(pickle_in)

def load_data(id_file):
    pickle_in = open("elmo_restaurant/absa16/"+ str(id_file).replace(':','_') +".pickle", "rb")
    return pickle.load(pickle_in)[0]

In [3]:
import numpy as np

def similarity(category, word):
    res = []
    for cat in category:
        res.append(cosine(cat, word))
    return np.mean(res)

In [4]:
def sim_result(word):
    c1 = similarity(ambience, word)
    c2 = similarity(food, word)
    c3 = similarity(service, word)
    c4 = similarity(value, word)
    return c1, c2, c3, c4

In [5]:
def cat_str(a, b, c, d):
    if a > b and a > c and a > d:
        return 1
    elif b > a and b > c and b > d:
        return 2
    elif c > b and c > a and c > d:
        return 3
    elif d > b and d > c and d > a:
        return 4
    else:
        return 5

In [6]:
def str_cat(num):
    pred =''
    if num == 1:
        pred = 'AMBIENCE'
    elif num == 2:
        pred = 'FOOD'
    elif num == 3:
        pred = 'SERVICE'
    elif num == 4:
        pred = 'PRICES'
    return pred

In [7]:
# service

In [8]:
ambience = load_category('ambience')
food = load_category('food')
service = load_category('service')
value = load_category('price')

import pandas as pd
from tqdm import tqdm_notebook

def run(tipe, tipe_keluar, where):
    df = pd.read_csv('Results/' + where + tipe +'.csv')
    sf = pd.DataFrame(columns=['id', 'review', 'target', 'label', 'predict', 'term', 'polarity'])
    for index in tqdm_notebook(range(len(df))):
        pred= ''
        pred_mult=[]
        if type(df['term'][index]) != float:
            terms = df['term'][index].lower().split('|')
            target = df['target'][index]
            id_file = df['id'][index]
            label = df['category'][index]
            review = df['review'][index]
            polarity = df['polarity'][index]

            vector = load_data(id_file)
            tokenized = df['review'][index].lower().replace('  ', ' ').strip().split(' ')

            for term in terms:
                i = 0
                for token in tokenized:
                    if term in token:
                        a, b, c, d = sim_result(vector[i])
                        x = cat_str(a, b, c, d)
                        pred_mult.append(str_cat(x))
                        break
                    i += 1
        else:
            terms = ''
            target = df['target'][index]
            id_file = df['id'][index]
            label = df['category'][index]
            polarity = df['polarity'][index]
            review = df['review'][index]
            
        sf = sf.append({'id': id_file,
                        'review': review,
                        'target': target,
                        'label': label,
                        'predict': '|'.join(pred_mult),
                        'term': '|'.join(terms),
                        'polarity': polarity
                       }, ignore_index=True)

        sf.to_csv("Results/Categorization/"+ tipe +tipe_keluar+".csv")
        sf.to_excel("Results/Categorization/"+ tipe +tipe_keluar+".xlsx")

In [11]:
run('lda2vec', '', 'Aspect Terms Extraction/')

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))

In [13]:
run('lda2vec-partial', '', 'add_noun/')

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))

In [15]:
run('lda2vec-partial-ner', '', 'Filter/')

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))

In [16]:
run('dp', '', 'Aspect Terms Extraction/')

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))

In [17]:
run('lda', '', 'Aspect Terms Extraction/')

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))

In [18]:
run('plsa', '', 'Aspect Terms Extraction/')

HBox(children=(IntProgress(value=0, max=464), HTML(value='')))